In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSE_Solver.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// set parameterz
int[] core_sweep = {128};
int[] PolyDegS = new int[] {3};
int[] ResArray = new int[] {15};
int MemoryPerCore = 2000;

In [ ]:
bool useAMR = true;
bool useLoadBal = true;
int NoOfTimeSteps = 60;
bool Steady = false;
bool IncludeConvection = true;
var Gshape = Shape.Cube;

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,BatchInstructionDir,AllowedDatabasesPaths,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,Password,PrivateKeyFilePath,AdditionalBatchCommands,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,MonoDebug
0,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\weber\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,<null>,,,,,,,,,,,,,
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\weber\dply,True,FDY-cluster,dotnet,,"[ { \\hpccluster\hpccluster-scratch\weber == : LocalMountPath: \\hpccluster\hpccluster-scratch\weber, PathAtRemote: } ]",FDY\weber,DC2,[ HPCCLUSTER2 ],Normal,True,,,,,,,
2,BoSSS.Application.BoSSSpad.SlurmClient,X:\Deployerie,True,<null>,dotnet,,,jw52xeqa,lcluster5.hrz.tu-darmstadt.de,,,,<null>,C:\Users\weber\.ssh\id_rsa,<null>,/home/jw52xeqa/Deployerie,project01217,<null>,False
3,BoSSS.Application.BoSSSpad.SlurmClient,X:\Deployerie,True,HHLR,dotnet,,"[ { W:\work\scratch\jw52xeqa == /work/scratch/jw52xeqa: LocalMountPath: W:\work\scratch\jw52xeqa, PathAtRemote: /work/scratch/jw52xeqa } ]",jw52xeqa,lcluster14.hrz.tu-darmstadt.de,,,,<null>,C:\Users\weber\.ssh\id_rsa,"[ #SBATCH -p test24, #SBATCH -C avx512 ]",/home/jw52xeqa/Deployerie,special00006,<null>,False


In [ ]:
// ==================================
// setup Client & Workflow & Database
// ==================================
var myBatch = (SlurmClient)ExecutionQueues[3];
var AddSbatchCmds = new List<string>();
AddSbatchCmds.AddRange(new string[]{"#SBATCH -p test24", "#SBATCH -C avx512", "#SBATCH --mem-per-cpu="+MemoryPerCore});
myBatch.AdditionalBatchCommands = AddSbatchCmds.ToArray();
myBatch.AdditionalBatchCommands

index,value
0,#SBATCH -p test24
1,#SBATCH -C avx512
2,#SBATCH --mem-per-cpu=2000


In [ ]:
//var myBatch = (MsHPC2012Client)ExecutionQueues[1];

In [ ]:
string WFlowName = "rotCube+Inlet_test";
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

In [ ]:
myBatch.AllowedDatabasesPaths.Clear();
//string dirname = "DB_rotCubePaper";
//string dirname = "DB_rotSphere_CoreScaling";
//string dirname = "DB_rotSphere_DOFScaling";
string dirname ="DB_rotCubePaper";
string winpath = @"W:\work\scratch\jw52xeqa\"+dirname;
string remotepath = @"/work/scratch/jw52xeqa/"+dirname;
//string winpath = @"\\hpccluster\hpccluster-scratch\"+dirname;
var myDB = OpenOrCreateDatabase(winpath);
//myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(winpath,remotepath)); myBatch.AllowedDatabasesPaths

Error: System.IO.DirectoryNotFoundException: Could not find a part of the path 'W:\work\scratch\jw52xeqa\DB_rotCubePaper'.
   at System.IO.FileSystem.CreateDirectory(String fullPath, Byte[] securityDescriptor)
   at System.IO.DirectoryInfo.Create()
   at BoSSS.Foundation.IO.DatabaseUtils.CreateDatabase(String dbDir) in B:\BoSSS-smuda\public\src\L2-foundation\BoSSS.Foundation\DatabaseUtils.cs:line 46
   at BoSSS.Application.BoSSSpad.InteractiveShell.OpenOrCreateDatabase_Impl(String dbDir, Boolean allowCreation) in B:\BoSSS-smuda\public\src\L4-application\BoSSSpad\InteractiveShell.cs:line 352
   at BoSSS.Application.BoSSSpad.BoSSSshell.OpenOrCreateDatabase(String dbDir) in B:\BoSSS-smuda\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 473
   at Submission#18.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 curren

In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
}

using 

In [ ]:
double xMax = 4.0, yMax = 1.0, zMax = 1.0;
double xMin = -2.0, yMin = -1.0,zMin = -1.0;

double xMax = 1.0, yMax = 1.0, zMax=1.0;
double xMin = -1.0, yMin = -1.0,zMin = -1.0;

In [ ]:
List<IGridInfo> Grids = new List<IGridInfo>();
foreach(var Res in ResArray){
    int Stretching = (int)Math.Floor(Math.Abs(xMax-xMin)/Math.Abs(yMax-yMin));
    var _xNodes = GenericBlas.Linspace(xMin, xMax, Stretching*Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);

    GridCommons grd;
    string gname = "RotBenchmarkGrid";
    
    var tmp = new List<IGridInfo>();
    foreach(var grid in myDB.Grids){
        try{
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells==_xNodes.Length*_yNodes.Length*_zNodes.Length;
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    if(tmp.Count()>=1){
        Console.WriteLine("Grid found: "+tmp.Pick(0).Name);
        Grids.Add(tmp.Pick(0));
        continue;
    }
    
    grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes, _zNodes);
    grd.Name = gname;
    //grd.AddPredefinedPartitioning("debug", MakeDebugPart);

    grd.EdgeTagNames.Add(1, "Velocity_inlet");
    grd.EdgeTagNames.Add(2, "Wall");
    grd.EdgeTagNames.Add(3, "Pressure_Outlet");

    grd.DefineEdgeTags(delegate (double[] _X) {
        var X = _X;
        double x, y, z;
        x = X[0];
        y = X[1];
        z = X[2];
        if(Math.Abs(x-xMin)<1E-8)
            return 1;
        else
            return 3;
    });
    myDB.SaveGrid(ref grd);
    Grids.Add(grd);
} Grids

Error: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#21.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [ ]:
int SpaceDim = 3;
double _partRad = 0.3800;

In [ ]:
_partRad/(2/(double)ResArray[0]/4)

11.4

In [ ]:
Func<IGridInfo, int, double, XNSE_Control> GenXNSECtrl = delegate(IGridInfo grd, int k, double Viscosity){
    XNSE_Control C = new XNSE_Control();
    // basic database options
    // ======================
    C.AlternateDbPaths = new[] {
        (winpath, ""),
        (remotepath,"")};
    //C.DbPath=@"\\hpccluster\hpccluster-scratch\DB_rotSphereBenchmark";
    C.savetodb = true;
    int J  = grd.NumberOfCells;
    C.SessionName = string.Format("J{0}_k{1}_t{2}", J, k,NoOfTimeSteps);
    if(IncludeConvection){
        C.SessionName += "_NSE";
        C.Tags.Add("NSE");
    } else {
        C.SessionName += "_Stokes";
        C.Tags.Add("Stokes");
    }
    C.Tags.Add(SpaceDim + "D");
    if(Steady)C.Tags.Add("steady");
    else C.Tags.Add("transient");

    // DG degrees
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 2));
    C.saveperiod = 1;
    //C.TracingNamespaces = "*";

    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.clusterHilbert;
    C.DynamicLoadbalancing_ClassifierType = ClassifierType.CutCells;
    C.DynamicLoadBalancing_On = useLoadBal;
    C.DynamicLoadBalancing_RedistributeAtStartup = true;
    C.DynamicLoadBalancing_Period = 1;
    C.DynamicLoadBalancing_ImbalanceThreshold = 0.1;


    // Physical Parameters
    // ===================
    const double rhoA = 1;
    const double Re = 100;
    double muA = Viscosity;
    
    double partRad = _partRad;
    double d_hyd = 2*partRad;
    double anglev = Re*muA/rhoA/d_hyd;
    double VelocityIn = Re*muA/rhoA/d_hyd;
    double[] pos = new double[SpaceDim];
    double ts = Math.PI/anglev/NoOfTimeSteps;
    double inletdelay = 5*ts;

    C.PhysicalParameters.IncludeConvection = IncludeConvection;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;

    C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim);
    C.Rigidbody.SpecifyShape(Gshape);
    C.Rigidbody.SetRotationAxis("x");
    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.UseImmersedBoundary = true;
    
    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Pressure_Outlet");
    //C.AddBoundaryValue("Velocity_inlet","VelocityX",new Formula($"(X,t) => {VelocityIn}*(double)(t<={inletdelay}?(t/{inletdelay}):1)",true));
    C.AddBoundaryValue("Velocity_inlet","VelocityX",new Formula($"(X) => {VelocityIn}"));

    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = 0.1;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None;
    C.Timestepper_LevelSetHandling = LevelSetHandling.Coupled_Once;
    C.LinearSolver.NoOfMultigridLevels = 5;
    C.LinearSolver.ConvergenceCriterion = 1E-6;
    C.LinearSolver.MaxSolverIterations = 200;
    C.LinearSolver.MaxKrylovDim = 50;
    C.LinearSolver.TargetBlockSize = 10000;
    C.LinearSolver.verbose = true;
    C.LinearSolver.SolverCode = LinearSolverCode.exp_Kcycle_schwarz;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 1E-3;
    C.NonLinearSolver.MaxSolverIterations = 6;
    C.NonLinearSolver.verbose = true;

    C.AdaptiveMeshRefinement = useAMR;
    if (useAMR) {
        C.SetMaximalRefinementLevel(2);
        C.AMR_startUpSweeps = 0;
    }

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = 1000;
        C.NoOfTimesteps = 1;
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = ts;        
        C.NoOfTimesteps = NoOfTimeSteps;
    }
    C.TimeSteppingScheme = TimeSteppingScheme.BDF2;
    C.dtFixed = dt;
    return C;
};

In [ ]:
var ViscositySweep = new double[]{1E-2};
List<XNSE_Control> controls = new List<XNSE_Control>();
foreach(IGridInfo grd in Grids){
    foreach(int k in PolyDegS){
        foreach(double v in ViscositySweep)
            controls.Add(GenXNSECtrl(grd,k,v));
    }
} controls.Select(s=>s.SessionName)

(empty)

In [ ]:
controls.Select(s=>s.SessionName)

(empty)

In [ ]:
static Func<int, int> NodeRegression = (int cores) => (int)Math.Ceiling(0.75*Math.Pow(cores,0.44));

In [ ]:
NodeRegression(128)

7

In [ ]:
foreach(var ctrl in controls){
    string sessname = ctrl.SessionName;
    foreach(int cores in core_sweep){
        ctrl.SessionName = sessname + "_c"+cores+"_mue_"+ctrl.PhysicalParameters.mu_A;
        var aJob   = new Job("rotting_"+Gshape+ctrl.SessionName,typeof(XNSE));
        aJob.SetControlObject(ctrl);
        aJob.NumberOfMPIProcs         = cores;
        aJob.ExecutionTime            = "3:00:00";
        aJob.UseComputeNodesExclusive = true;

        List<string> Cmdtmp = myBatch.AdditionalBatchCommands.ToList();
        int NoOfNodes = 4;
        Cmdtmp.Add($"#SBATCH -N {NoOfNodes}");
        myBatch.AdditionalBatchCommands = Cmdtmp.ToArray();

        aJob.Activate(myBatch);
    }
}